## Train and test FastText with LDA labels and then predict class for test set
Split doc2vec training into labelled train and test set

In [ ]:
from sklearn.cross_validation import train_test_split
X = xrange(len(docs))
y = xrange(len(docs))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
train_file = codecs.open('/home/daniel/Desktop/fastText/train.txt', 'w', 'utf-8')
for i in X_train:
    label =  ' __label__Topic' + str(np.argmax(topics_dist[i,:])) + '\n'
    train_file.write(docs[i].strip() + label)
train_file.close()

test_file = codecs.open('/home/daniel/Desktop/fastText/test.txt', 'w', 'utf-8')
for i in X_test:
    label =  ' __label__Topic' + str(np.argmax(topics_dist[i,:])) + '\n'
    test_file.write(docs[i].strip() + label)
test_file.close()

In [ ]:
%%script bash
cd ~/ms/Daniel/fastText/
./fasttext supervised -dim 150 -lr 0.1 -wordNgrams 3 -input ~/Desktop/fastText/train.txt -output ~/Desktop/fastText/train

In [ ]:
%%script bash
cd ~/ms/Daniel/fastText/
./fasttext test ~/Desktop/fastText/train.bin ~/Desktop/fastText/test.txt

### Train FastText model

In [ ]:
%%script bash
cd ~/ms/Daniel/fastText
#./fasttext skipgram -dim 150 -minCount 1 -input /home/daniel/s2ds/Data/04_doc2vecTrainingDataFiltered.txt -output /home/daniel/Desktop/fastText/model
./fasttext supervised -dim 100 -input /home/daniel/s2ds/Data/05_fastTextConvLabelled.txt -output /home/daniel/Desktop/fastText/modelLabel
#./fasttext supervised -dim 100 -wordNgrams 3 -input /home/daniel/s2ds/Data/05_clientMessagesFilteredfastTextLabelled.txt -output /home/daniel/Desktop/fastText/modelLabel

### Load the fastText model and make a (row) normalized word embedding from it

In [ ]:
import codecs, os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
%cd ~/Desktop/fastText/
word_vects = pd.read_table('modelLabel.vec', index_col=0, header=None, sep=' ')   
# last col is all nans, because an added extra space, get rid off it
word_vects = word_vects.iloc[:, :word_vects.shape[1]-1]
# normalize all word vects to one
# word_norms = np.linalg.norm(word_vects.values, axis=1)
# data = word_vects.values / word_norms[:, np.newaxis]
# word_vects = pd.DataFrame(data, index=word_vects.index)

check if the word embedding make sence, by looking for closest vectors in the vector space

In [ ]:
word_norms = np.linalg.norm(word_vects.values, axis=1)
data = word_vects.values / word_norms[:, np.newaxis]
word_vects = pd.DataFrame(data, index=word_vects.index)
def get_nearest_vect(word):
    query = word_vects.loc[word].values/np.linalg.norm(word_vects.loc[word].values)
    cosines = np.dot(word_vects, query)
    # get second closest vector (becuase the closest will be the query)
    sorted_cosines = np.argsort(cosines)
    cos_counter = 0
    for i, cos in enumerate(cosines[sorted_cosines][::-1]):
        if cos_counter == 1 and cos != np.nan:
            # tricky indexing because we reversed the sorted list above
            max_cosine = sorted_cosines[-i-1]
            break
        if not np.isnan(cos):
            cos_counter += 1

    return word_vects.index[max_cosine], cosines[max_cosine]

In [ ]:
get_nearest_vect('order')
#np.linalg.norm(word_vects.loc['receive'])

### Train TF-IDF on data doc2vecTraingingData and collapse it down to a dict

In [ ]:
import codecs, os
import numpy as np
import pandas as pd
data_folder = '/home/daniel/s2ds/Data/'
input_file = '05_fastTextConv.txt'
input_file = codecs.open(os.path.join(data_folder, input_file), 'rU', 'utf-8')
docs = []
for line in input_file:
    docs.append(line.strip())

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=.95, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(docs)
tfidf_words = tfidf_vectorizer.get_feature_names()
tfidf = np.mean(tfidf.toarray(), axis=0)
tfidf_dict = {}
for i, t in enumerate(tfidf):
    tfidf_dict[tfidf_words[i]] = t

In [ ]:
import operator
sorted_x = sorted(tfidf_dict.items(), key=operator.itemgetter(1))
#sorted_x[-100:]

## Generate simple client vectors with averaging

In [ ]:
client_file = '05_clientMessagesFilteredfastText.txt'
client_file = codecs.open(os.path.join(data_folder, client_file), 'rU', 'utf-8')
summary_file = os.path.join(data_folder, 'client_agent_summary3.csv')
summary = pd.read_csv(summary_file, index_col=0)
# get the position of each sentence in a given conversation: matches length of client_file
sent_pos = summary.convPos[~pd.isnull(summary.linePosFiltered)].values

In [ ]:
def get_sent_comp_vect2(embedding_df, sent):
    sent = sent.split()
    sent_vect = None
    w_count = 0
    for word in sent:
        try:
            word_vect = embedding_df.loc[word].values
            if sent_vect is None:
                sent_vect = word_vect
            else:
                np.add(sent_vect, word_vect)
            w_count += 1
        except:
            pass
    # return vect with zeros if None
    if sent_vect is None:
        return np.zeros(embedding_df.shape[1])
    else:
        # an averaged vector
        return sent_vect/w_count

In [ ]:
desktop = '/home/daniel/Desktop/fastText/'
client_sentences = 'clientMessagesVectors_Labelled_nomemory.csv'
client_sentences = codecs.open(os.path.join(desktop, client_sentences), 'w', 'utf-8')
# keeping track of the current conversation
prev_sent_pos = 100
# prev sentences' weigh less
past_w = .7
curr_w = 1
pos_counter = 1
for i, line in enumerate(client_file):
    if i % 10000 == 0:
        print i
    sent_vect = get_sent_comp_vect2(word_vects, line)
    line = str(i) + ',' + ','.join(map(str, sent_vect)) + '\n'
    client_sentences.write(line) 
    prev_sent_pos = sent_pos[i]

### Generate time-aware sentence vectors from the filtered client corpus

In [ ]:
def get_sent_comp_vect(tfidf_dict, embedding_df, sent):
    sent = sent.split()
    sent_vect = None
    w_count = 0
    for word in sent:
        try:
            word_vect = embedding_df.loc[word].values
            word_w = tfidf_dict[word]
            if sent_vect is None:
                sent_vect = word_vect * word_w
            else:
                np.add(sent_vect, word_vect * word_w)
            w_count += 1
        except:
            pass
    # return vect with zeros if None
    if sent_vect is None:
        return np.zeros(embedding_df.shape[1])
    else:
        # an averaged vector
        return sent_vect/w_count

In [ ]:
desktop = '/home/daniel/Desktop/fastText/'
client_sentences = 'clientMessagesVectors_Labelled.csv'
client_sentences = codecs.open(os.path.join(desktop, client_sentences), 'w', 'utf-8')
# keeping track of the current conversation
prev_sent_pos = 100
# prev sentences' weigh less
past_w = .7
curr_w = 1
pos_counter = 1
for i, line in enumerate(client_file):
    if i % 10000 == 0:
        print i
    sent_vect = get_sent_comp_vect(tfidf_dict, word_vects, line)
    # new conversation
    if sent_pos[i] < prev_sent_pos:
        tmp_sent_vect = sent_vect
        pos_counter = 1
    else:
        # add curr sent vect to previous sentences  
        tmp_sent_vect = np.add(tmp_sent_vect * past_w, sent_vect * curr_w) / pos_counter
        pos_counter += 1
    tmp_sent_vect /= np.linalg.norm(tmp_sent_vect)
    line = str(i) + ',' + ','.join(map(str, tmp_sent_vect)) + '\n'
    client_sentences.write(line) 
    prev_sent_pos = sent_pos[i]

## SHOWTIME, Chatty2

Load sentence vectors

In [ ]:
client_vects = pd.read_csv('/home/daniel/Desktop/fastText/clientMessagesVectors_Labelled.csv', index_col=0, header=None)  

Define input cleaning function

In [ ]:
%cd ~/ms/Daniel/
import preprocessing.filtering as filtering
reload(filtering)
from preprocessing.filtering import NLTKPreprocessor
from nltk.corpus import words
data_folder = '/home/daniel/s2ds/Data/'
# nltk_preproc = NLTKPreprocessor()
names = filtering.build_names(data_folder)
# list of words for spellcheck
word_dict = {w: 1 for w in words.words()}
contractions = {v: k for k, v in filtering.get_contractions().items()}
word_dict.update(contractions)
def clean_input(sentence):
    sent_clean = filtering.filter_line(sentence, preprocessing=True, names=names, spellcheck=word_dict)
    return sent_clean

Make lookup sentence DataFrame

In [ ]:
from itertools import izip
data_folder = '/home/daniel/s2ds/Data/'
client_file = '03_clientMessages.txt'
client_file = codecs.open(os.path.join(data_folder, client_file), 'rU', 'utf-8')
agent_file = '03_agentMessages.txt'
agent_file = codecs.open(os.path.join(data_folder, agent_file), 'rU', 'utf-8')
client_agent_dict = {}
line_count = 0
with client_file as client, agent_file as agent:
        for client_line, agent_line in izip(client, agent):
            client_line = client_line.strip()
            agent_line = agent_line.strip()
            client_agent_dict[line_count] = (client_line, agent_line) 
            line_count += 1

In [ ]:
# instead of KDtree, we normalize the sentence vectors to length 1 and take 
# the dotproduct with the sentence matrix and find the smallest cosine similarity
client_norms = np.linalg.norm(client_vects.values, axis=1)
data = client_vects.values / client_norms[:, np.newaxis]
client_vects_norm = pd.DataFrame(data, index=client_vects.index)
def get_closest_cosine(query_vect):
    query_vect /= np.linalg.norm(query_vect)
    cosines = np.dot(client_vects_norm.values, query_vect)
    return np.nanargmax(cosines)

In [ ]:
from sklearn.neighbors import KDTree
tree = KDTree(client_vects)

In [ ]:
query = "I cannot find my parcel are you sure it was delivered at the right address?"
#query = 'I lost my password and now I cannot log in to my account '
#query = "Hi could you help me with a lost order?"
query_clean = clean_input(query)
print query_clean

query_vect = get_sent_comp_vect(tfidf_dict, word_vects, query_clean).reshape(1, -1)
#closest_filtered_ind = tree.query(query_vect, k=1)[1][0][0]
closest_filtered_ind = get_closest_cosine(query_vect.T)
# get the corresponding closest line in the unfiltered messages
line_num = np.where(summary.linePosFiltered.values == float(closest_filtered_ind))[0][0]
line_pos = summary.convPos.values[line_num]
line_num = summary.linePos.values[line_num]
print line_pos, client_agent_dict[line_num]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def chatbot():
    while True:
        query_clean = clean_input(raw_input())
        query_vect = get_sent_comp_vect(tfidf_dict, word_vects, query_clean).reshape(1, -1)
        closest_filtered_ind = get_closest_cosine(query_vect.T)
        #closest_filtered_ind = tree.query(query_vect, k=1)[1][0][0]
        # get the corresponding closest line in the unfiltered messages
        line_num = np.where(summary.linePosFiltered.values == float(closest_filtered_ind))[0][0]
        line_num = summary.linePos.values[line_num]
        print client_agent_dict[line_num]
chatbot()

### Fit LDA to doc2vecTrainingData and print most common words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import codecs, os
import numpy as np
import pandas as pd

data_folder = '/home/daniel/s2ds/Data/'
input_file = '05_fastTextConv.txt'
input_file = codecs.open(os.path.join(data_folder, input_file), 'rU', 'utf-8')
docs = []
for line in input_file:
    docs.append(line.strip())
    
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

#lda = LatentDirichletAllocation(n_topics=21, learning_offset=20., random_state=0, n_jobs=-1, max_iter=30)
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=.01, max_features=3000, stop_words='english')
tf = tf_vectorizer.fit_transform(docs)
#lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
print_top_words(lda, tf_feature_names, 12)

Save LDA model

In [ ]:
from sklearn.externals import joblib
joblib.dump(lda, '/home/daniel/Desktop/fastText/lda.pkl') 

In [2]:
from sklearn.externals import joblib
lda = joblib.load('/home/daniel/Desktop/fastText/lda.pkl')

Predict the topic dist for all conversations

In [6]:
%timeit topics_dist = lda.transform(tf[1,:])

1 loops, best of 3: 207 ms per loop


Create training data for fastText classifier from 05_fastTextCov.txt

In [ ]:
# from nltk.corpus import stopwords as sw
# from nltk import wordpunct_tokenize
# stopwords = set(sw.words('english'))
output_file = codecs.open(os.path.join(data_folder, '05_fastTextConvLabelled.txt'), 'w', 'utf-8')
for i, doc in enumerate(docs):
    label =  ' __label__Topic' + str(np.argmax(topics_dist[i,:])) + '\n'
    # doc_filtered = ' '.join([w for w in wordpunct_tokenize(doc.strip()) if w not in stopwords])
    output_file.write(doc.strip() + label)
output_file.close()

Create training data for fastText classifier from 05_clientMessagesFiltered.txt

In [ ]:
input_file = codecs.open(os.path.join(data_folder, '05_clientMessagesFilteredfastText.txt'), 'rU', 'utf-8')
output_file = codecs.open(os.path.join(data_folder, '05_clientMessagesFilteredfastTextLabelled.txt'), 'w', 'utf-8')
summary_file = os.path.join(data_folder, 'client_agent_summary3.csv')
summary = pd.read_csv(summary_file, index_col=0)
# get the position of each sentence in a given conversation: matches length of client_file
sent_pos = summary.convPos[~pd.isnull(summary.linePosFiltered)].values
for i, line in enumerate(input_file):
    line_num = np.where(summary.linePosFiltered.values == float(i))[0]
    conv_id = int(summary.convIDFiltered.values[line_num])
    label =  ' __label__Topic' + str(np.argmax(topics_dist[conv_id,:])) + '\n'
    output_file.write(line.strip() + label)
output_file.close()